# Joining RDD's

In [3]:
filename = '../data/babynames.csv'
if not os.path.exists(filename):
    import urllib.request
    urllib.request.urlretrieve ("https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv?accessType=DOWNLOAD", \
                                filename)

In [4]:
def readCSV(fname, header=False, separator=','):
    rdd = sc.textFile(fname)
    if header:
        firstline = rdd.first()
        rdd = rdd.filter(lambda x: x != firstline)
    return rdd.map(lambda x: x.split(separator))

babyrdd = readCSV(filename, header=True)
print(babyrdd.take(5))

[['2013', 'GAVIN', 'ST LAWRENCE', 'M', '9'], ['2013', 'LEVI', 'ST LAWRENCE', 'M', '9'], ['2013', 'LOGAN', 'NEW YORK', 'M', '44'], ['2013', 'HUDSON', 'NEW YORK', 'M', '49'], ['2013', 'GABRIEL', 'NEW YORK', 'M', '50']]


Some names are given to both girls and boys, let us list those names, together with how often a boy received that name and how often a girl.

To start, per gender, we generate an RDD of (first name, frequency), and use reduceByKey to aggregate those names over all regions and years.

## Join

In two (key, value) RDDs A and B, we can use the (inner) **join()** transformation to obtain all possible combinations for which A.key == B.key. The result is pair of the form (key, (value1, value2)).

In [6]:
boys = babyrdd.filter(lambda x: x[3] == 'M').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)
girls = babyrdd.filter(lambda x: x[3] == 'F').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)
        
samename = boys.join(girls)
print(samename.collect())

[('LOGAN', (5203, 24)), ('TAYLOR', (17, 1638)), ('DYLAN', (6419, 219)), ('JAIME', (29, 7)), ('HAYDEN', (412, 119)), ('KAMARI', (44, 6)), ('CAMERON', (2278, 87)), ('SKYLER', (77, 42)), ('JORDAN', (2854, 272)), ('EDEN', (31, 245)), ('JAYLIN', (30, 90)), ('ROYAL', (5, 5)), ('QUINN', (162, 258)), ('HUNTER', (2052, 12)), ('RYAN', (8995, 68)), ('TENZIN', (188, 180)), ('EMERSON', (80, 91)), ('SHILOH', (18, 38)), ('AMARI', (345, 23)), ('SKYLAR', (5, 785)), ('JAIDEN', (806, 6)), ('JAYDEN', (9836, 33)), ('AVERY', (395, 1918)), ('ARIEL', (490, 533)), ('PHOENIX', (33, 44)), ('FINLEY', (6, 5)), ('AUGUST', (190, 5)), ('LONDON', (5, 939)), ('MILAN', (70, 121)), ('BLAKE', (1195, 51)), ('PARIS', (5, 208)), ('YAEL', (6, 109)), ('ROWAN', (66, 5)), ('SHEA', (72, 12)), ('PEYTON', (200, 1036)), ('SAGE', (5, 19)), ('KAI', (529, 5)), ('RILEY', (571, 1610)), ('ALEXIS', (573, 1779)), ('CHARLIE', (231, 25)), ('REESE', (5, 273)), ('ANGEL', (3148, 220)), ('CASEY', (58, 130)), ('PARKER', (766, 30)), ('EMERY', (7, 1

## LeftOuterJoin

Alternatively, two (key, value) RDD's A and B can be combined with a **leftOuterJoin** (or rightOuterJoin), resulting in an RDD that contains all (key, value) pairs in A, and when available joined by matching (key, value) pairs in B. If no matching (key, value) pair is found in B, the value for B is set to *None*.

NB there is also a `fullOuterJoin` and a `rightOuterJoin`.

In [22]:
boys = babyrdd.filter(lambda x: x[3] == 'M' and x[2] == 'NEW YORK').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)
girls = babyrdd.filter(lambda x: x[3] == 'F' and x[2] == 'NEW YORK').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)

samename = boys.leftOuterJoin(girls)
print(samename.collect())

[('AIDEN', (395, None)), ('ELLIOT', (50, None)), ('ETHAN', (723, None)), ('JUAN', (81, None)), ('ALLEN', (23, None)), ('LOGAN', (201, None)), ('EITAN', (10, None)), ('DEVIN', (108, None)), ('GABRIEL', (410, None)), ('PRESTON', (12, None)), ('ISAAC', (248, None)), ('KAIDEN', (12, None)), ('JORGE', (10, None)), ('PEDRO', (22, None)), ('MARTIN', (35, None)), ('ANDY', (86, None)), ('JEFFREY', (24, None)), ('QUINN', (23, 23)), ('LUKE', (307, None)), ('WESLEY', (28, None)), ('FREDERICK', (11, None)), ('AUSTIN', (115, None)), ('ISAIAH', (151, None)), ('AVERY', (36, 218)), ('VINCENT', (165, None)), ('ANTHONY', (321, None)), ('CODY', (12, None)), ('LUCAS', (547, None)), ('KADEN', (10, None)), ('ARJUN', (13, None)), ('JASON', (263, None)), ('RAPHAEL', (51, None)), ('JESUS', (65, None)), ('REX', (12, None)), ('ROBERT', (219, None)), ('RAYMOND', (58, None)), ('JUDE', (43, None)), ('KIERAN', (34, None)), ('PATRICK', (114, None)), ('DYLAN', (552, 93)), ('SHANE', (25, None)), ('JAKE', (237, None)), (

## Cogroup

A more powerful way to control a join between two (key, value) RDDs A and B, is to use the **cogroup** transformation. The resulting RDD will contain a full outer join of A and B of the form `(key, (<values in A iter>, <values in B iter>))`.

Note that cogroup returns Iterable objects, which you can turn into lists using the list function.

In [23]:
boys = babyrdd.filter(lambda x: x[3] == 'M' and x[2] == 'NEW YORK' and x[0] == '2013').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)
girls = babyrdd.filter(lambda x: x[3] == 'F' and x[2] == 'NEW YORK' and x[0] == '2013').\
        map(lambda x: (x[1], int(x[4]))).\
        reduceByKey(lambda x, y: x + y)

samename = boys.cogroup(girls).map(lambda x: (x[0], (list(x[1][0]), list(x[1][1]))))
print(samename.collect())

[('AIDEN', ([53], [])), ('AARON', ([38], [])), ('EVAN', ([26], [])), ('JAMES', ([99], [])), ('COLETTE', ([], [10])), ('DECLAN', ([11], [])), ('JESSICA', ([], [11])), ('NICOLAS', ([21], [])), ('DEVIN', ([15], [])), ('GABRIEL', ([50], [])), ('VIVIENNE', ([], [23])), ('SKYLAR', ([], [14])), ('JOSEPHINE', ([], [13])), ('KAIDEN', ([12], [])), ('IRIS', ([], [12])), ('BRYAN', ([10], [])), ('VICTOR', ([13], [])), ('KAYLA', ([], [14])), ('ZOEY', ([], [20])), ('KIARA', ([], [11])), ('QUINN', ([], [12])), ('JAYDEN', ([66], [])), ('CHRISTOPHER', ([52], [])), ('TRISTAN', ([20], [])), ('ARIA', ([], [17])), ('ETHAN', ([97], [])), ('LUKE', ([41], [])), ('STEVEN', ([13], [])), ('WESLEY', ([14], [])), ('NORA', ([], [21])), ('LAYLA', ([], [19])), ('AUSTIN', ([22], [])), ('MICHELLE', ([], [10])), ('BRADY', ([12], [])), ('RYAN', ([92], [])), ('GABRIELLA', ([], [28])), ('CAYDEN', ([10], [])), ('JASMINE', ([], [13])), ('AMBER', ([], [14])), ('VINCENT', ([19], [])), ('ANTHONY', ([33], [])), ('EDWARD', ([20], 